In [1]:
using Pkg
using DotEnv
include("src/download.jl")


download_product (generic function with 1 method)

In [2]:
DotEnv.config();
username = get(ENV, "DATASPACE_USERNAME", "");
password = get(ENV, "DATASPACE_PASSWORD", "");
access_token = get_access_token(get(ENV, "DATASPACE_USERNAME", ""), get(ENV, "DATASPACE_PASSWORD", ""));

MethodError: MethodError: no method matching get_access_token(::String, ::String)

In [33]:
data = download_sentinel_metadata(product_type= "GRD",verbose=1);
df = vector_of_dicts_to_dataframe(data["features"]);

Request successful.


In [3]:
access_token

UndefVarError: UndefVarError: access_token not defined

In [72]:
println(df)

8×51 DataFrame
 Row │ properties_polarisation  properties_license_viewService  properties_status  properties_swath  properties_centroid_type  properties_license_hasToBeSigned  properties_updated        properties_productType  properties_platform  properties_license_description_shortName  properties_parentIdentifier  properties_instrument  properties_license_grantedFlags  properties_description             properties_productIdentifier       properties_resolution  properties_completionDate  properties_links_1_title           properties_links_1_type  geometry_type  properties_services_download_url   properties_services_download_mimeType  properties_orbitDirection  properties_title                   properties_thumbnail               geometry_coordinates_1             properties_license_licenseId  properties_cloudCover  id                                 properties_links_1_rel  properties_processingLevel  properties_gmlgeometry             properties_centroid_coordinates_2  properties_star

In [55]:
df.properties_links_1_title[1]

"GeoJSON link for 8bb78fee-2441-57d2-b020-2e951450ec9b"

In [73]:
df.id[1]

"8bb78fee-2441-57d2-b020-2e951450ec9b"

In [59]:
df.properties_links_1_href[1]

"https://catalogue.dataspace.copernicus.eu/resto/collections/SENTINEL-1/8bb78fee-2441-57d2-b020-2e951450ec9b.json"

In [ ]:
https://catalogue.dataspace.copernicus.eu/resto/collections/SENTINEL-1/8bb78fee-2441-57d2-b020-2e951450ec9b.json"

In [58]:
df.properties_services_download_url[1]

"https://catalogue.dataspace.copernicus.eu/download/8bb78fee-2441-57d2-b020-2e951450ec9b"

In [62]:
curl -o te.zip "https://catalogue.dataspace.copernicus.eu/resto/collections/SENTINEL-1/8bb78fee-2441-57d2-b020-2e951450ec9b.json"

ErrorException: syntax: extra token """ after end of expression

In [57]:
df.properties_productIdentifier[1]

"/eodata/Sentinel-1/SAR/GRD/2022/06/21/S1A_IW_GRDH_1SDV_20220621T055905_20220621T055930_043757_053958_45B9.SAFE"

Download billederne

In [71]:
using HTTP
using ProgressMeter
using HTTP
using FileIO
function download_product(access_token::String, product_id::String, filename::String)
    # URL of the file
    url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products($product_id)/\$value"

    # Authorization header
    headers = Dict("Authorization" => "Bearer $access_token")

    try
        # Make an HTTP GET request with the specified headers
        response = HTTP.get(url, headers=headers, request_timeout=30)
        
        # Check if the response status is successful
        if HTTP.status(response) == 200
            # Specify the filename to save the downloaded file
            # Open the file for writing
            open(filename, "w") do file
                # Write the content of the response to the file
                write(file, response.body)
            end
            println("File downloaded successfully.")
        else
            println("Failed to download the file. Status code: ", HTTP.status(response))
        end
    catch e
        println("Error: ", e)
    end

end



download_product (generic function with 1 method)

In [76]:


output_file = "product2.zip"

download_product(get_access_token(get(ENV, "DATASPACE_USERNAME", ""), get(ENV, "DATASPACE_PASSWORD", "")), df.id[2], "product3.zip")

File downloaded successfully.


In [109]:
using ZipFile
using FileIO
using Glob

function unzip_files2(;input_path::AbstractString, output_folder::AbstractString)
    try
        # Create the output folder if it doesn't exist
        mkpath(output_folder)

        if isfile(input_path) && endswith(input_path, ".zip")
            # If input_path is a single ZIP file, extract it
            unzip_file(input_path, output_folder)
        elseif isdir(input_path)
            # If input_path is a directory, unzip all ZIP files in it
            zip_files = glob("*.zip", input_path)
            for zip_file in zip_files
                unzip_file(joinpath(input_path, zip_file), output_folder)
            end
        else
            println("Error: Invalid input path. Please provide either a single ZIP file or a folder containing ZIP files.")
        end
    catch e
        println("Error: $e")
    end
end

# Example usage:
# unzip_files("path/to/your/archive.zip", "path/to/output/folder")
# unzip_files("path/to/your/folder", "path/to/output/folder")


unzip_files2 (generic function with 2 methods)

In [110]:
unzip_files2("/Users/kaaso/Documents/coding/JuliaEO2024/product3.zip", "/Users/kaaso/Documents/coding/JuliaEO2024")

Error: ErrorException("type ReadableFile has no field iscompressed")


In [ ]:
/Users/kaaso/Documents/coding/JuliaEO2024/product3.zip

In [111]:
using ZipFile
import Glob

# Function to unzip a single file
function unzip_file(zip_path::String, output_folder::String)
    z = ZipFile.Reader(zip_path)
    try
        for f in z.files
            # No need to check if the file is compressed
            out_path = joinpath(output_folder, f.name)
            mkpath(dirname(out_path))
            write(out_path, read(f))
        end
    finally
        close(z)
    end
end

# Function to unzip all files in a folder
function unzip_all(folder_path::String, output_folder::String)
    zip_files = Glob.glob("*.zip", folder_path)
    for zip_file in zip_files
        full_path = joinpath(folder_path, zip_file)
        unzip_file(full_path, output_folder)
    end
end

# Example Usage:
# To unzip a single file:
# unzip_file("path/to/your/file.zip", "path/to/output/folder")

# To unzip all files in a folder:
# unzip_all("path/to/your/folder", "path/to/output/folder")


unzip_all (generic function with 1 method)

In [112]:
unzip_file("/Users/kaaso/Documents/coding/JuliaEO2024/product3.zip", "/Users/kaaso/Documents/coding/JuliaEO2024/")


SystemError: SystemError: opening file "/Users/kaaso/Documents/coding/JuliaEO2024/S1A_IW_GRDH_1SDV_20220619T172458_20220619T172523_043735_0538B0_1641.SAFE/": Is a directory

In [87]:
unzip_file("/Users/kaaso/Documents/coding/JuliaEO2024/product3.zip")

Unzipping: /Users/kaaso/Documents/coding/JuliaEO2024/product3.zip
Extracting: /Users/kaaso/Documents/coding/JuliaEO2024/S1A_IW_GRDH_1SDV_20220619T172458_20220619T172523_043735_0538B0_1641.SAFE/


SystemError: SystemError: opening file "/Users/kaaso/Documents/coding/JuliaEO2024/S1A_IW_GRDH_1SDV_20220619T172458_20220619T172523_043735_0538B0_1641.SAFE/": No such file or directory

In [ ]:
S1A_IW_GRDH_1SDV_20220619T172458_20220619T172523_043735_0538B0_1641

In [47]:
using HTTP
using ProgressMeter

function download_product(access_token::String, product_id::String, output_file::String)
    url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products($(product_id))/\$value"
    headers = Dict("Authorization" => "Bearer $access_token")

    response = HTTP.get(url, headers=headers; stream=true)

    # Convert headers to a dictionary and try to get the total size
    headers_dict = Dict(response.headers)
    total_size = parse(Int, get(headers_dict, "Content-Length", "-1"))

    # Initialize the progress bar, handle case when total_size is unknown (-1)
    p = total_size > 0 ? Progress(total_size, 1, "Downloading $(product_id): ", 60) : nothing

    open(output_file, "w") do file
        downloaded_size = 0
        for chunk in HTTP.payload(response)
            write(file, chunk)
            downloaded_size += length(chunk)
            # Update the progress bar if total_size is known
            if total_size > 0
                ProgressMeter.update!(p, downloaded_size)
            end
        end
    end

    if total_size <= 0
        println("Download completed: $product_id")
    end
end

# Example usage
product_id = "8bb78fee-2441-57d2-b020-2e951450ec9b" # Replace with actual product ID
output_file = "product2.zip"

download_product(access_token, product_id, output_file)


HTTP.Exceptions.StatusError: HTTP.Exceptions.StatusError(401, "GET", "/odata/v1/Products(8bb78fee-2441-57d2-b020-2e951450ec9b)/\$value", HTTP.Messages.Response:
"""
HTTP/1.1 401 Unauthorized
Date: Mon, 13 Nov 2023 17:07:18 GMT
Content-Type: application/json
Content-Length: 31
Connection: keep-alive
x-envoy-upstream-service-time: 8
Strict-Transport-Security: max-age=15724800; includeSubDomains
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET, PUT, POST, DELETE, PATCH, OPTIONS
Access-Control-Allow-Headers: DNT,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range,Authorization
Access-Control-Max-Age: 1728000

{"detail":"Expired signature!"}""")

In [65]:
import Pkg; Pkg.add("FileIO")

   Resolving package versions...


    Updating `~/Documents/coding/JuliaEO2024/Project.toml`
  [5789e2e9] + FileIO v1.16.1
  No Changes to `~/Documents/coding/JuliaEO2024/Manifest.toml`


In [69]:
using HTTP
using FileIO

# URL of the file
url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products(8bb78fee-2441-57d2-b020-2e951450ec9b)/\$value"

# Authorization header
headers = Dict("Authorization" => "Bearer $access_token")

try
    # Make an HTTP GET request with the specified headers
    response = HTTP.get(url, headers=headers, request_timeout=30)
    
    # Check if the response status is successful
    if HTTP.status(response) == 200
        # Specify the filename to save the downloaded file
        filename = "product_del2.zip"
        
        # Open the file for writing
        open(filename, "wb") do file
            # Write the content of the response to the file
            foreach(chunk -> write(file, chunk), response.body)
        end
        
        println("File downloaded successfully.")
    else
        println("Failed to download the file. Status code: ", HTTP.status(response))
    end
catch e
    println("Error: ", e)
end


Error: ArgumentError("invalid open mode: wb")


In [70]:
using HTTP
using FileIO

# URL of the file
url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products(8bb78fee-2441-57d2-b020-2e951450ec9b)/\$value"

# Authorization header
headers = Dict("Authorization" => "Bearer $access_token")

try
    # Make an HTTP GET request with the specified headers
    response = HTTP.get(url, headers=headers, request_timeout=30)
    
    # Check if the response status is successful
    if HTTP.status(response) == 200
        # Specify the filename to save the downloaded file
        filename = "product_del2.zip"
        
        # Open the file for writing
        open(filename, "w") do file
            # Write the content of the response to the file
            write(file, response.body)
        end
        
        println("File downloaded successfully.")
    else
        println("Failed to download the file. Status code: ", HTTP.status(response))
    end
catch e
    println("Error: ", e)
end


File downloaded successfully.


In [52]:
using HTTP
using HTTP
using ProgressMeter

function download_product(access_token::String, product_id::String, output_file::String)
    url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products($(product_id))/\$value"
    headers = Dict("Authorization" => "Bearer $access_token")

    response = HTTP.get(url, headers=headers; stream=true)

    # Convert headers to a dictionary and get the total size of the file
    headers_dict = Dict(response.headers)
    total_size = parse(Int, get(headers_dict, "Content-Length", "0"))

    # Initialize the progress bar
    p = Progress(total_size, 1, "Downloading $(product_id): ", 60)

    open(output_file, "w") do file
        for chunk in HTTP.payload(response)
            write(file, chunk)
            # Update the progress bar
            ProgressMeter.update!(p, length(chunk))
        end
    end
end


#product_id = "8bb78fee-2441-57d2-b020-2e951450ec9b" # Replace with actual product ID
#output_file = "product.zip"

download_product(acces_token, df.id[6], output_file)


HTTP.Exceptions.StatusError: HTTP.Exceptions.StatusError(401, "GET", "/odata/v1/Products(4c496182-e715-51cc-8da2-b4d16edd05e2)/\$value", HTTP.Messages.Response:
"""
HTTP/1.1 401 Unauthorized
Date: Mon, 13 Nov 2023 17:08:07 GMT
Content-Type: application/json
Content-Length: 31
Connection: keep-alive
x-envoy-upstream-service-time: 9
Strict-Transport-Security: max-age=15724800; includeSubDomains
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET, PUT, POST, DELETE, PATCH, OPTIONS
Access-Control-Allow-Headers: DNT,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range,Authorization
Access-Control-Max-Age: 1728000

{"detail":"Expired signature!"}""")

"http://codede.data.cloud.code-de.org/Sentinel-1/SAR/GRD/2022/06/21/S1A_IW_GRDH_1SDV_20220621T055905_20220621T055930_043757_053958_45B9.SAFE/preview/quick-look.png"

In [24]:
using Pkg
Pkg.add("ProgressMeter")


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


    Updating `~/Documents/coding/JuliaEO2024/Project.toml`
  [92933f4c] + ProgressMeter v1.9.0
    Updating `~/Documents/coding/JuliaEO2024/Manifest.toml`
  [92933f4c] + ProgressMeter v1.9.0
  [8ba89e20] + Distributed


In [34]:
using HTTP
using ProgressMeter


# Incorrect way that might cause the error
headers = [("Authorization", "Bearer $access_token")] # This is a vector of pairs
header_value = get(headers, "Authorization", "default_value") # This will cause an error

# Correct way if headers is a dictionary
headers = Dict("Authorization" => "Bearer $access_token")
header_value = get(headers, "Authorization", "default_value") # This is correct




function download_product(access_token::String, product_id::String, output_file::String)
    url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products($(product_id))/\$value"
    headers = Dict("Authorization" => "Bearer $access_token")

    response = HTTP.get(url, headers=headers; stream=true)

    # Determine the total size of the file
    total_size = parse(Int, get(response.headers, "Content-Length", "0"))
    
    # Initialize the progress bar
    p = Progress(total_size, 1, "Downloading $(product_id): ", 60)

    open(output_file, "w") do file
        for chunk in HTTP.payload(response)
            write(file, chunk)
            # Update the progress bar
            ProgressMeter.update!(p, length(chunk))
        end
    end
end

# Example usage
#access_token = "your_access_token" # Replace with your actual access token
product_id = "8c4ecdb7-079a-534d-a398-24354168dcd4" # Replace with actual product ID
output_file = "product_test.zip"

download_product(access_token, product_id, output_file)


MethodError: MethodError: no method matching get(::Vector{Pair{SubString{String}, SubString{String}}}, ::String, ::String)
Closest candidates are:
  get(!Matched::DataFrames.DataFrameColumns, ::Union{AbstractString, Signed, Symbol, Unsigned}, ::Any) at ~/.julia/packages/DataFrames/58MUJ/src/abstractdataframe/iteration.jl:314
  get(!Matched::Base.ImmutableDict, ::Any, ::Any) at dict.jl:821
  get(!Matched::DataStructures.OrderedRobinDict{K, V}, ::Any, ::Any) where {K, V} at ~/.julia/packages/DataStructures/MKv4P/src/ordered_robin_dict.jl:261
  ...

In [39]:
using Images

In [42]:
using HTTP
using Images

# URL of the image
url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(19ba6517-a22f-5e18-8fce-67af19ce0faa)/\$value"
url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(8c4ecdb7-079a-534d-a398-24354168dcd4)/\$value"
url = df.properties_thumbnail[1]

try
    # Download the image
    response = HTTP.get(url)
    
    if HTTP.statusok(response)
        # Read the image data
        img_data = HTTP.body(response)
        
        # Create an image object from the data
        img = Images.load(IOBuffer(img_data))
        
        # Display the image
        display(img)
    else
        println("Failed to fetch the image. Status code: ", HTTP.status(response))
    end
catch e
    println("Error: ", e)
end


Error: HTTP.Exceptions.ConnectError("http://codede.data.cloud.code-de.org/Sentinel-1/SAR/GRD/2022/06/21/S1A_IW_GRDH_1SDV_20220621T055905_20220621T055930_043757_053958_45B9.SAFE/preview/quick-look.png", CapturedException(DNSError: codede.data.cloud.code-de.org, unknown node or service (EAI_NONAME), Any[(getalladdrinfo(host::String) at addrinfo.jl:113, 1), (getalladdrinfo at addrinfo.jl:122 [inlined], 1), (getconnection(::Type{Sockets.TCPSocket}, host::SubString{String}, port::SubString{String}; keepalive::Bool, readtimeout::Int64, kw::Base.Pairs{Symbol, Union{Nothing, Integer}, NTuple{4, Symbol}, NamedTuple{(:require_ssl_verification, :iofunction, :decompress, :verbose), Tuple{Bool, Nothing, Nothing, Int64}}}) at Connections.jl:515, 1), (#10 at Connections.jl:462 [inlined], 1)

, (macro expansion at try_with_timeout.jl:82 [inlined], 1), ((::ConcurrentUtilities.var"#2#4"{Any, Timer, ConcurrentUtilities.TimedOut{Any}, Channel{Any}, HTTP.Connections.var"#10#13"{Sockets.TCPSocket, Bool, Bool, Base.Pairs{Symbol, Union{Nothing, Int64}, NTuple{4, Symbol}, NamedTuple{(:readtimeout, :iofunction, :decompress, :verbose), Tuple{Int64, Nothing, Nothing, Int64}}}, SubString{String}, SubString{String}}})() at threadingconstructs.jl:258, 1)]))


In [ ]:
"https://catalogue.dataspace.copernicus.eu/odata/v1/Assets($(df.id[2]))/\$value"

In [25]:
"https://catalogue.dataspace.copernicus.eu/odata/v1/Assets($(df.id[2]))/\$value"

"https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(19ba6517-a22f-5e18-8fce-67af19ce0faa)/\$value"

In [26]:
curl -o quick-look.png "https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(19ba6517-a22f-5e18-8fce-67af19ce0faa)/$value"


"8c4ecdb7-079a-534d-a398-24354168dcd4"

In [ ]:
"https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(19ba6517-a22f-5e18-8fce-67af19ce0faa)/\$value"

In [ ]:
https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(19ba6517-a22f-5e18-8fce-67af19ce0faa)/$value
